In [68]:
import pandas as pd

data : pd.DataFrame = pd.read_csv('Zomato-data-.csv')
data.head()

,name,online_order,book_table,rate,votes,approx_cost(for two people),listed_in(type)
0,Jalsa,Yes,Yes,4.1/5,775,800,Buffet
1,Spice Elephant,Yes,No,4.1/5,787,800,Buffet
2,San Churro Cafe,Yes,No,3.8/5,918,800,Buffet
3,Addhuri Udupi Bhojana,No,No,3.7/5,88,300,Buffet
4,Grand Village,No,No,3.8/5,166,600,Buffet


In [69]:
rate_data:pd.Series = data['rate']

for i in range(len(rate_data)):
    rate_data.iloc[i] = rate_data.iloc[i].split('/')[0]

# rate_data

C:\Users\Admin1\AppData\Local\Temp\ipykernel_328\650526796.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rate_data.iloc[i] = rate_data.iloc[i].split('/')[0]


In [70]:
df = data.drop(['rate'],axis=1)

In [71]:
df['rate'] = rate_data
df['rate'] = df['rate'].astype('float')
df.head()

,name,online_order,book_table,votes,approx_cost(for two people),listed_in(type),rate
0,Jalsa,Yes,Yes,775,800,Buffet,4.1
1,Spice Elephant,Yes,No,787,800,Buffet,4.1
2,San Churro Cafe,Yes,No,918,800,Buffet,3.8
3,Addhuri Udupi Bhojana,No,No,88,300,Buffet,3.7
4,Grand Village,No,No,166,600,Buffet,3.8


In [72]:
df1 = df.drop('name', axis=1)

In [73]:
df1.head()

,online_order,book_table,votes,approx_cost(for two people),listed_in(type),rate
0,Yes,Yes,775,800,Buffet,4.1
1,Yes,No,787,800,Buffet,4.1
2,Yes,No,918,800,Buffet,3.8
3,No,No,88,300,Buffet,3.7
4,No,No,166,600,Buffet,3.8


In [74]:
df1['online_order'].replace({'Yes':1, 'No':0},inplace=True)
df1['online_order'].astype('int8')
df1['book_table'].replace({'Yes':1, 'No':0},inplace=True)
df1['book_table'].astype('int8')
df1.head()

,online_order,book_table,votes,approx_cost(for two people),listed_in(type),rate
0,1,1,775,800,Buffet,4.1
1,1,0,787,800,Buffet,4.1
2,1,0,918,800,Buffet,3.8
3,0,0,88,300,Buffet,3.7
4,0,0,166,600,Buffet,3.8


In [75]:
df1.votes = df1.votes.astype('int32')
df1['approx_cost(for two people)'] = df1['approx_cost(for two people)'].astype('int16')

In [76]:
df['listed_in(type)'].unique()

array(['Buffet', 'Cafes', 'other', 'Dining'], dtype=object)

In [77]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

df = pd.DataFrame({
    'listed_in(type)': ['Buffet', 'Cafes', 'other', 'Dining', 'Buffet']
})

preprocessor = ColumnTransformer(
    transformers=[
        ('listed_in', OneHotEncoder(sparse=False), ['listed_in(type)'])
    ],
    remainder='passthrough'  # Keep other columns as is
)

df_encoded = preprocessor.fit_transform(df)

encoded_df = pd.DataFrame(df_encoded, columns=preprocessor.get_feature_names_out())
# print(encoded_df)

c:\Users\Admin1\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [78]:
df1.columns

Index(['online_order', 'book_table', 'votes', 'approx_cost(for two people)',
       'listed_in(type)', 'rate'],
      dtype='object')

In [79]:
encoded_df.columns

Index(['listed_in__listed_in(type)_Buffet', 'listed_in__listed_in(type)_Cafes',
       'listed_in__listed_in(type)_Dining',
       'listed_in__listed_in(type)_other'],
      dtype='object')

In [80]:
encoded_df.columns = [col.replace('listed_in__listed_in(type)_', '') for col in encoded_df.columns]

df1 = df1.drop(columns=['listed_in(type)'])

df1_encoded = pd.concat([df1, encoded_df], axis=1)

df1_encoded.head()

,online_order,book_table,votes,approx_cost(for two people),rate,Buffet,Cafes,Dining,other
0,1,1,775,800,4.1,1.0,0.0,0.0,0.0
1,1,0,787,800,4.1,0.0,1.0,0.0,0.0
2,1,0,918,800,3.8,0.0,0.0,0.0,1.0
3,0,0,88,300,3.7,0.0,0.0,1.0,0.0
4,0,0,166,600,3.8,1.0,0.0,0.0,0.0


In [81]:
df1_encoded['Buffet'].isna().count()

148

In [82]:
df1_encoded[['Buffet', 'Cafes', 'Dining', 'other']] = df1_encoded[['Buffet', 'Cafes', 'Dining', 'other']].fillna(0)
df1_encoded[['Buffet', 'Cafes', 'Dining', 'other']] = df1_encoded[['Buffet', 'Cafes', 'Dining', 'other']].astype('int8')
print(df1_encoded)

     online_order  book_table  votes  approx_cost(for two people)  rate  \
0               1           1    775                          800   4.1   
1               1           0    787                          800   4.1   
2               1           0    918                          800   3.8   
3               0           0     88                          300   3.7   
4               0           0    166                          600   3.8   
..            ...         ...    ...                          ...   ...   
143             0           0      0                          100   3.3   
144             0           0      0                          150   3.3   
145             1           0    771                          450   4.0   
146             0           0     98                          800   3.0   
147             1           0     47                          200   3.9   

     Buffet  Cafes  Dining  other  
0         1      0       0      0  
1         0      1       0 

In [83]:
x = df1_encoded.drop('rate',axis=1)
x.head()

,online_order,book_table,votes,approx_cost(for two people),Buffet,Cafes,Dining,other
0,1,1,775,800,1,0,0,0
1,1,0,787,800,0,1,0,0
2,1,0,918,800,0,0,0,1
3,0,0,88,300,0,0,1,0
4,0,0,166,600,1,0,0,0


In [84]:
y = df1_encoded['rate']
y.head()

0    4.1
1    4.1
2    3.8
3    3.7
4    3.8
Name: rate, dtype: float64

In [105]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split


lr = LinearRegression()
x_train,x_test ,y_train,y_test = train_test_split(x,y,test_size=0.2)
lr.fit(x_train,y_train)
lr.score(x_test,y_test)

0.25853135958069995

In [110]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression

kf = KFold(n_splits=5, shuffle=True, random_state=42)

lr = LinearRegression()

scores = []

for train_index, test_index in kf.split(x, y):
    x_train_fold, x_test_fold = x.iloc[train_index], x.iloc[test_index]
    y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]
    
    lr.fit(x_train_fold, y_train_fold)
    
    scores.append(lr.score(x_test_fold, y_test_fold))

print("Cross-validation scores:", scores)


Cross-validation scores: [0.1801525984925677, 0.34108186458968803, 0.499918660982392, 0.15455283304359269, 0.2930918146389333]


In [111]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LinearRegression

kf = KFold(n_splits=5, shuffle=True, random_state=42)

lr = LinearRegression()

scores = []

for train_index, test_index in kf.split(x, y):
    x_train_fold, x_test_fold = x.iloc[train_index], x.iloc[test_index]
    y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]
    
    lr.fit(x_train_fold, y_train_fold)
    
    scores.append(lr.score(x_test_fold, y_test_fold))

print("Cross-validation scores:", scores)

Cross-validation scores: [0.1801525984925677, 0.34108186458968803, 0.499918660982392, 0.15455283304359269, 0.2930918146389333]
